In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 90.2 MB/s eta 0:00:00


In [1]:
from pulp import *
from sklearn.metrics.pairwise import haversine_distances
from scipy.spatial.distance import cdist
from random import sample
import numpy as np
from pulp import LpVariable
import pandas as pd
## Calculate distance between two points
from math import radians, cos, sin, asin, sqrt
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus
from re import L

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
plants = pd.read_csv('./data_source/Location for plant - plant.csv')
demand = pd.read_excel('./data_source/Regional Demand Distribution.xlsx' , sheet_name = 'Sheet1')
zone = pd.read_excel('./data_source/Regional Demand Distribution.xlsx' , sheet_name = 'zone_state')
warehouse = pd.read_csv('./data_source/Exhibit 5.csv')
sec_freight = pd.read_excel('./data_source/exhibit_3.xlsx', sheet_name = 'Sheet1')

In [ ]:
demand.head(10)

,Region,Demand
0,SW,75690
1,W,168200
2,SE,134560
3,NC,134560
4,NW,92510
5,E,117740
6,NE,25230
7,C,84100
8,N,8410


In [ ]:
warehouse.head()

,Warehouse,Latitude,Longtitude,City,State
0,W1,27.59137,76.43122,Roondh Rampur,Rajasthan
1,W2,26.85383,84.19225,Amwas Khas,Uttar Pradesh
2,W3,17.20569,81.53633,Rajahmundry,Andhra Pradesh
3,W4,28.56098,80.49032,Sumer Nagar,Uttar Pradesh
4,W5,29.25725,77.91551,Bhoomma,Uttar Pradesh


In [ ]:
zone.head()

,Zone,States,Latitude,Longtitude,Total Demand,CNO Demand,VAHO Demand,Edible Demand
0,North,"[Srinagar, Himachal Pradesh]",30.319034,78.838999,8410,3868.6,2018.4,2523
1,North Central,"[Rajasthan, Haryana, Uttar Pradesh]",28.772754,78.413475,134560,61897.6,32294.4,40368
2,Central,"[Madhya Pradesh, Chhattisgarh]",22.873469,78.641025,84100,38686.0,20184.0,25230
3,West,[Maharashtra],19.547596,75.396701,168200,77372.0,40368.0,50460
4,South West,"[Karnataka, Kerala]",11.986364,76.456848,75690,34817.4,18165.6,22707


In [ ]:
plants.head()

,Plant,Latitude,Longtitude,Intercept,Slope,Production
0,Guwuhati,26.115284,91.708029,2.73,0.0020,VAHO
1,Dehradun,30.316215,78.032581,3.84,0.0052,VAHO
2,Baddi,30.957890,76.790862,3.84,0.0052,Edible
3,Jalgaon,21.007706,75.563233,2.52,0.0048,Edible
4,Kanjikode,10.796767,76.748803,1.64,0.0042,CNO


In [ ]:
### define several parameters
total_warehousing_rental_cost = 9360000  # for 24 warehouses
average_rent_per_warehouse = total_warehousing_rental_cost / 24
budget_for_rental = 8580000  # for 22 warehouses
max_number_of_open_warehouses = budget_for_rental / average_rent_per_warehouse

## Function

> **Function of distance calculation**

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

> **Calculate primary freight cost and secondary freight cost**

In [ ]:
### calculate the distances
primary_freight_cno = {}
primary_freight_vaho = {}
primary_freight_edible = {}
for p, p_data in plants.iterrows():
    for w, w_data in warehouse.iterrows():
        if p_data['Production'] == 'CNO':
          # print(p_data['Longtitude'],p_data['Latitude'],w_data['Longtitude'], w_data['Latitude'])
          distance = haversine(p_data['Longtitude'], p_data['Latitude'], w_data['Longtitude'], w_data['Latitude'])
          # print(distance)
          cost = p_data['Intercept'] + p_data['Slope'] * distance
          primary_freight_cno[(p_data['Plant'], w_data['Warehouse'])] = cost
        elif p_data['Production'] == 'VAHO':
          distance = haversine(p_data['Longtitude'], p_data['Latitude'], w_data['Longtitude'], w_data['Latitude'])
          cost = p_data['Intercept'] + p_data['Slope'] * distance
          primary_freight_vaho[(p_data['Plant'], w_data['Warehouse'])] = cost
        else:
          distance = haversine(p_data['Longtitude'], p_data['Latitude'], w_data['Longtitude'], w_data['Latitude'])
          cost = p_data['Intercept'] + p_data['Slope'] * distance
          primary_freight_edible[(p_data['Plant'], w_data['Warehouse'])] = cost

secondary_freight_cno = {}
secondary_freight_vaho = {}
secondary_freight_edible = {}

for w, w_data in warehouse.iterrows():
    for z, z_data in zone.iterrows():
        distance = haversine(w_data['Longtitude'], w_data['Latitude'], z_data['Longtitude'], z_data['Latitude'])
        secondary_freight_cno[(w_data['Warehouse'], z_data['Zone'])] = distance * sec_freight[sec_freight['Zone'] == z_data["Zone"]]['Coefficient']
        secondary_freight_vaho[(w_data['Warehouse'], z_data['Zone'])] = distance * sec_freight[sec_freight['Zone'] == z_data["Zone"]]['Coefficient']
        secondary_freight_edible[(w_data['Warehouse'], z_data['Zone'])] = distance * sec_freight[sec_freight['Zone'] == z_data["Zone"]]['Coefficient']

In [ ]:

def warehouse_optimizer(num_warehouses, zone_df , print_log = False):
    # Select random warehouses
    # selected_warehouses = sample(warehouse.index.tolist(), num_warehouses)
    # selected_warehouse_df = warehouse.loc[selected_warehouses]
    selected_warehouse_df = warehouse
    zone = zone_df
    # Variables
    open_warehouse = LpVariable.dicts("", selected_warehouse_df['Warehouse'], lowBound=0, upBound=1, cat='Integer')

    flow_p2w_cno = LpVariable.dicts("R1_flow_p2w_cno", [(p['Plant'], w['Warehouse']) for _, p in plants_cno.iterrows() for _, w in selected_warehouse_df.iterrows()], lowBound=0, cat='Integer')
    flow_p2w_vaho = LpVariable.dicts("R1_flow_p2w_vaho", [(p['Plant'], w['Warehouse']) for _, p in plants_vaho.iterrows() for _, w in selected_warehouse_df.iterrows()], lowBound=0, cat='Integer')
    flow_p2w_edible = LpVariable.dicts("R1_flow_p2w_edible", [(p['Plant'], w['Warehouse']) for _, p in plants_edible.iterrows() for _, w in selected_warehouse_df.iterrows()], lowBound=0, cat='Integer')

    flow_w2z_cno = LpVariable.dicts("R2_flow_w2z_cno", [(w['Warehouse'], z['Zone']) for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()], lowBound=0, cat='Integer')
    flow_w2z_vaho = LpVariable.dicts("R2_flow_w2z_vaho", [(w['Warehouse'], z['Zone']) for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()], lowBound=0, cat='Integer')
    flow_w2z_edible = LpVariable.dicts("R2_flow_w2z_edible", [(w['Warehouse'], z['Zone']) for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()], lowBound=0, cat='Integer')

    # Objective Function
    prob = LpProblem("Warehouse_Distribution_Problem", LpMinimize)
    prob += (
              lpSum([primary_freight_cno.get((p['Plant'], w['Warehouse']), 0) * flow_p2w_cno.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_cno.iterrows() for _, w in selected_warehouse_df.iterrows()]) +
              lpSum([primary_freight_vaho.get((p['Plant'], w['Warehouse']), 0) * flow_p2w_vaho.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_vaho.iterrows() for _, w in selected_warehouse_df.iterrows()]) +
              lpSum([primary_freight_edible.get((p['Plant'], w['Warehouse']), 0) * flow_p2w_edible.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_edible.iterrows() for _, w in selected_warehouse_df.iterrows()]) +
              lpSum([secondary_freight_cno.get((w['Warehouse'], z['Zone']), 0) * flow_w2z_cno.get((w['Warehouse'], z['Zone']), 0)for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()])+
              lpSum([secondary_freight_vaho.get((w['Warehouse'], z['Zone']), 0) * flow_w2z_vaho.get((w['Warehouse'], z['Zone']), 0) for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()])+
              lpSum([secondary_freight_edible.get((w['Warehouse'], z['Zone']), 0) * flow_w2z_edible.get((w['Warehouse'], z['Zone']), 0) for _, w in selected_warehouse_df.iterrows() for _, z in zone.iterrows()]))

    # Constraints
    ### 1. Make sure the inflow = outflow for warehouse (v2)
    for _, w in selected_warehouse_df.iterrows():
        prob += lpSum([flow_p2w_cno.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_cno.iterrows()]) == lpSum([flow_w2z_cno.get((w['Warehouse'], z['Zone']), 0) for _, z in zone.iterrows()])
        prob += lpSum([flow_p2w_vaho.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_vaho.iterrows()]) == lpSum([flow_w2z_vaho.get((w['Warehouse'], z['Zone']), 0) for _, z in zone.iterrows()])
        prob += lpSum([flow_p2w_edible.get((p['Plant'], w['Warehouse']), 0) for _, p in plants_edible.iterrows()]) == lpSum([flow_w2z_edible.get((w['Warehouse'], z['Zone']), 0) for _, z in zone.iterrows()])

    ### 2. Make sure the demand is fulfilled
    for _, z in zone.iterrows():
        prob += (lpSum([flow_w2z_cno.get((w['Warehouse'], z['Zone']), 0) for _, w in selected_warehouse_df.iterrows()]) >= z['CNO Demand'])
        prob += (lpSum([flow_w2z_vaho.get((w['Warehouse'], z['Zone']), 0) for _, w in selected_warehouse_df.iterrows()]) >= z['VAHO Demand'])
        prob += (lpSum([flow_w2z_edible.get((w['Warehouse'], z['Zone']), 0) for _, w in selected_warehouse_df.iterrows()]) >= z['Edible Demand'])

    ### 3. Make sure the warehouses are open 
    M = 1000000
    for _, w in selected_warehouse_df.iterrows():
        prob += lpSum(flow_p2w_cno.get((p['Plant'], w['Warehouse']) for _, p in plants_cno.iterrows())) <= M * open_warehouse[w['Warehouse']]
        prob += lpSum(flow_p2w_vaho.get((p['Plant'], w['Warehouse'])) for _, p in plants_vaho.iterrows()) <= M * open_warehouse[w['Warehouse']]
        prob += lpSum(flow_p2w_edible.get((p['Plant'], w['Warehouse']) for _, p in plants_edible.iterrows())) <= M * open_warehouse[w['Warehouse']]

    ### 4. Make sure the distances from warehouses to zone do not exceed 500km (2 service days)
    for _, z in zone.iterrows():
        prob += lpSum([open_warehouse[w['Warehouse']] for _, w in selected_warehouse_df.iterrows() if haversine(w['Longtitude'], w['Latitude'], z['Longtitude'], z['Latitude']) <= 500]) >= 1

    ### 5. Ensure that we select a certain number of warehouses 
    prob += lpSum([open_warehouse[w] for w in warehouse['Warehouse']]) == num_warehouses

    # Solve the problem
    prob.solve()

    # Print the status of the problem
    print(f'Status: {LpStatus[prob.status]}')

    # Print the optimal total cost
    total_cost = value(prob.objective) #+ num_warehouses * average_rent_per_warehouse
    print(f'Total cost: {total_cost}')

    if print_log:
        print("Variable values:")
        if prob.status == LpStatusOptimal:  # Check if the problem has an optimal solution
          for v in prob.variables():
              if v.varValue > 0:
                  print(f"{v.name}: {v.varValue}")
        else:
            print("No optimal solution found. The problem may be infeasible.")


    # Return the total cost for comparisons
    return total_cost

In [ ]:
cost_22_1 = warehouse_optimizer(22, zone)
cost_24_1 = warehouse_optimizer(24, zone)

Status: Optimal
Total cost: 5714492.277041137
Status: Optimal
Total cost: 5714492.277041137


In [ ]:
cost_22 = warehouse_optimizer(22 , zone , print_log = True)

Status: Optimal
Total cost: 5714492.277041137
Variable values:
R1_flow_p2w_cno_('Kanjikode',_'W17'): 61898.0
R1_flow_p2w_cno_('Kanjikode',_'W19'): 61898.0
R1_flow_p2w_cno_('Kanjikode',_'W26'): 42555.0
R1_flow_p2w_cno_('Kanjikode',_'W37'): 38686.0
R1_flow_p2w_cno_('Kanjikode',_'W43'): 34818.0
R1_flow_p2w_cno_('Kanjikode',_'W44'): 77372.0
R1_flow_p2w_cno_('Kanjikode',_'W5'): 3869.0
R1_flow_p2w_cno_('Pondicherry',_'W15'): 54161.0
R1_flow_p2w_cno_('Pondicherry',_'W27'): 11606.0
R1_flow_p2w_edible_('Baddi',_'W5'): 40368.0
R1_flow_p2w_edible_('Baddi',_'W52'): 2523.0
R1_flow_p2w_edible_('Jalgaon',_'W15'): 35322.0
R1_flow_p2w_edible_('Jalgaon',_'W18'): 22707.0
R1_flow_p2w_edible_('Jalgaon',_'W26'): 27753.0
R1_flow_p2w_edible_('Jalgaon',_'W27'): 7569.0
R1_flow_p2w_edible_('Jalgaon',_'W37'): 25230.0
R1_flow_p2w_edible_('Jalgaon',_'W43'): 40368.0
R1_flow_p2w_edible_('Jalgaon',_'W8'): 50460.0
R1_flow_p2w_vaho_('Dehradun',_'W5'): 32295.0
R1_flow_p2w_vaho_('Dehradun',_'W52'): 2019.0
R1_flow_p2w_vaho

### Elasticity of Demand Comparison

In [ ]:
zone.columns

Index(['Zone', 'States', 'Latitude', 'Longtitude', 'Total Demand',
       'CNO Demand', 'VAHO Demand', 'Edible Demand'],
      dtype='object')

In [ ]:
def demand_changes(N, zone):
  df = zone
  df['CNO Demand'] = df['CNO Demand'] * N
  df['VAHO Demand'] = df['VAHO Demand'] * N
  df['Edible Demand'] = df['Edible Demand'] * N

  return df

In [ ]:
print(zone)

            Zone                                             States  \
0          North                       [Srinagar, Himachal Pradesh]   
1  North Central                [Rajasthan, Haryana, Uttar Pradesh]   
2        Central                     [Madhya Pradesh, Chhattisgarh]   
3           West                                      [Maharashtra]   
4     South West                                [Karnataka, Kerala]   
5     South East  [Andhra Pradesh, Telangana, Tamil Nadu , Andam...   
6           East            [Bihar, Jharkhand, West Bengal, Odisha]   
7     North East                                   [Assam, Manipur]   
8     North West                                          [Gujarat]   

    Latitude  Longtitude  Total Demand  CNO Demand  VAHO Demand  Edible Demand  
0  30.319034   78.838999          8410      3868.6       2018.4           2523  
1  28.772754   78.413475        134560     61897.6      32294.4          40368  
2  22.873469   78.641025         84100     386

In [ ]:
### see the changes of the costs
cost_22_1 = warehouse_optimizer(22, zone)

Status: Optimal
Total cost: 5714492.277041137


In [ ]:
zone2 = demand_changes(1.1, zone)
cost_22_2 = warehouse_optimizer(22, zone2)

Status: Optimal
Total cost: 6285983.27005686


In [ ]:
### calculate the elasticity
elasticity_22 = (cost_22_2 - cost_22_1) / cost_22_1
print(f'The elasiticity is {elasticity_22}')

The elasiticity is 0.10000730866533436
